In [1]:
import json
import numpy as np
from transformer_lens import HookedTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, log_loss
import torch
model = HookedTransformer.from_pretrained("gpt2")

def process_data(data_file):
    """Process prompts from JSON file and extract activation vectors from middle 50% of layers."""
    with open(data_file, "r") as f:
        data = json.load(f)
    
    activation_vectors = []
    labels = []
    
    for entry in data:
        # Process angry prompt (label = 1)
        angry_prompt = entry["angry"]
        tokens = model.to_tokens(angry_prompt)
        _, cache = model.run_with_cache(tokens)
        n_layers = model.cfg.n_layers
        start = n_layers // 4
        end = 3 * n_layers // 4
        mid_vec = torch.cat(
            [cache["resid_post", layer][0, -1] for layer in range(start, end)]
        ).cpu().numpy()
        activation_vectors.append(mid_vec)
        labels.append(1)
        
        # Process neutral prompt (label = 0)
        neutral_prompt = entry["neutral"]
        tokens = model.to_tokens(neutral_prompt)
        _, cache = model.run_with_cache(tokens)
        start = n_layers // 4
        end = 3 * n_layers // 4
        mid_vec = torch.cat(
            [cache["resid_post", layer][0, -1] for layer in range(start, end)]
        ).cpu().numpy()
        activation_vectors.append(mid_vec)
        labels.append(0)
    
    return np.stack(activation_vectors), np.stack(labels)

/home/bazzite/richard/gpt2-probe/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
`torch_dtype` is deprecated! Use `dtype` instead!


Loaded pretrained model gpt2 into HookedTransformer


In [2]:
# Load and process training data
print("Processing training data...")
train_vectors, train_labels = process_data("data/train.json")

# Load and process test data
print("Processing test data...")
test_vectors, test_labels = process_data("data/test.json")

# Save activation vectors and labels
np.save("train_activation_vectors.npy", train_vectors)
np.save("train_labels.npy", train_labels)
np.save("test_activation_vectors.npy", test_vectors)
np.save("test_labels.npy", test_labels)

# Train probe on training data
print("Training probe...")


Processing training data...
Processing test data...
Training probe...


In [3]:
probe = LogisticRegression(max_iter=5000).fit(train_vectors, train_labels)
train_predictions = probe.predict(train_vectors)
train_accuracy = accuracy_score(train_labels, train_predictions)
print(f"Train Accuracy: {train_accuracy}")
train_probabilities = probe.predict_proba(train_vectors)
train_loss = log_loss(train_labels, train_probabilities)
print(f"Train Log Loss: {train_loss}")

Train Accuracy: 1.0
Train Log Loss: 6.672037126153417e-05


In [4]:
# Evaluate on test data
print("Evaluating on test data...")
test_predictions = probe.predict(test_vectors)
test_accuracy = accuracy_score(test_labels, test_predictions)
print(f"Test Accuracy: {test_accuracy}")

# Get predicted probabilities for loss calculation
test_probabilities = probe.predict_proba(test_vectors)
test_loss = log_loss(test_labels, test_probabilities)
print(f"Test Log Loss: {test_loss}")

Evaluating on test data...
Test Accuracy: 1.0
Test Log Loss: 0.00014196169656475219
